# Uso de vectores semánticos en un clasificador *zero-shot*
Vamos a usar el vector semántico de los documentos para realizar una clasificación sobre unas categorías definidas semánticamente

In [1]:
import pandas as pd
import numpy as np

El dataset de textos a clasificar es un listado de compras, y cada categoría está definida por un listado de palabras.

In [2]:
#cargamos compras y clases
compras = pd.read_csv("compras_subset.csv", index_col=False)

clases = []
with open('clases.txt') as f:
    clases = [line[:-1] for line in f]
    
clases

['alimentos comida bebida carne pollo jugo',
 'alcohol cigarrillo tabaco',
 'ropa de vestir calzado zapatos vestidos',
 'muebles hogar aseo herramienta',
 'salud medicamento hospital',
 'transporte bus avión automóvil',
 'comunicaciones teléfono celular']

In [ ]:
compras.sample(5)

Cargamos los *word embeddings* del modelo FastText

In [ ]:
#carga de vectores en formato TXT
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = '~/Downloads/fasttext-sbwc.100k.vec'
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec)

Definimos dos funciones para calcular la simitud semántica

In [ ]:
from numpy.linalg import norm

def to_vector(texto):
    """Función para calcular vector semántico
    de un documento"""
    tokens = texto.lower().split()
    vec = np.zeros(300)
    for word in tokens:
        # si la palabra está la acumulamos
        if word in wordvectors:
            vec += wordvectors[word]
    return vec / norm(vec)

def similarity(texto_1, texto_2):
    """Calcula la similitud semántica de dos textos"""
    vec_1 = to_vector(texto_1)
    vec_2 = to_vector(texto_2)
    sim = vec_1 @ vec_2
    return sim

In [ ]:
similarity("cielo al amanecer azul", "las nubes amenazan lluvia")

In [ ]:
similarity("cielo al amanecer azul", "jugador de fútbol")

Usamos la similitud semántica para asignar cada compra a una clase:

In [ ]:
muestra = compras.sample(1)
texto = muestra["compra"].values[0]
clase_real = muestra["clase"].values[0]

sims = np.array([similarity(texto, clase) for clase in clases])
clase_pred = np.argmax(sims)

print('detalle compra:\t', texto)
print('clase predicha:\t', clases[clase_pred])
print('clase real:\t', clases[clase_real])

Calcular uno a uno la similitud con esta función es muy lento, lo hacemos vectorizando los corpus de compras y clases, y aplicando un producto matricial sobre estas matrices.

In [ ]:
# pon todos los vectores de compras en una matriz
compras_vectores = [to_vector(texto) for texto in compras["compra"]]
X = np.vstack(compras_vectores)

# pon todos los vectores de clases en una matriz
clases_vectores = [to_vector(cls) for cls in clases]
Y = np.vstack(clases_vectores)

print('X.shape =',X.shape)
print('Y.shape =',Y.shape)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

#calcula las similitudes como un producto punto
similitudes = X @ Y.T

pred = np.argmax(similitudes, axis=1)

print(classification_report(compras["clase"], pred))
print("Accuracy:", accuracy_score(compras["clase"], pred))